In [1]:
import os
import re

In [2]:
sorted([item for item in os.listdir() if os.path.isdir(item)])

['nltk-mfq', 'vanilla-mfq']

In [3]:
dataset_name = "vanilla-mfq"

In [4]:
trec_executable = "../trec_eval/trec_eval"     # path to the TREC executable

qrels_file_path = "../qrels/qrels_without_empty_nodes_file.txt"

run_output_folder = f"{dataset_name}/output"      # folder that contains the output of the runs
trec_output_folder = f"{dataset_name}/trec"       # folder that will contain the TREC output 

## TREC
Generate TREC output files

In [5]:
output_run = os.listdir(run_output_folder)

In [6]:
trec_cmd = f"{trec_executable} -q -m all_trec -c {qrels_file_path}"

In [7]:
shell_commands = list()
trec_output_files = list()

for file in output_run:
    trec_output_file = file.replace("output", "trec")
    cmd = f"{trec_cmd} {run_output_folder}/{file} > {trec_output_folder}/{trec_output_file}"

    trec_output_files.append(trec_output_file)
    shell_commands.append(cmd)

In [8]:
for cmd in shell_commands:
    os.system(cmd)

## Results
Extrapolate data from trec output

In [9]:
files = os.listdir(trec_output_folder)
files.sort()

In [10]:
# measurements = ["ndcg_cut_5", "ndcg_cut_10", "map_cut_5", "map_cut_10", "ndcg", "map" ,"set_recall"]
measurements = ["ndcg_cut_5", "ndcg_cut_10", "map_cut_5", "map_cut_10"]

def get_metrics(file_path: str)->dict:
    d = dict()
    with open(file_path) as f:
        for line in f:
            for m in measurements:
                if re.match(f"{m}\s", line):
                    d[m] = line.split()[-1]
    return d


In [11]:
out = dict()
for file in files:
    file_path = f"{trec_output_folder}/{file}"
    key = file.replace('-trec.txt','')
    out[key] =  get_metrics(file_path)

In [12]:
import pandas as pd

df = pd.DataFrame(out).transpose()
df.index.name = 'Model'
df.columns.name = 'Metric'

markdown_table = df.to_markdown()
print(markdown_table)

| Model                                  |   ndcg_cut_5 |   ndcg_cut_10 |   map_cut_5 |   map_cut_10 |
|:---------------------------------------|-------------:|--------------:|------------:|-------------:|
| BM25-EXTRACTED+NODES-ALL-QUERIES       |       0.1275 |        0.1413 |      0.0751 |       0.0891 |
| BM25-EXTRACTED+NODES-SYN-QUERIES       |       0.0736 |        0.0797 |      0.0415 |       0.0493 |
| BM25-EXTRACTED+NODES-TREC-QUERIES      |       0.0539 |        0.0616 |      0.0336 |       0.0398 |
| BM25-EXTRACTED-ALL-QUERIES             |       0.1056 |        0.1158 |      0.0653 |       0.0759 |
| BM25-EXTRACTED-SYN-QUERIES             |       0.0626 |        0.068  |      0.0367 |       0.0433 |
| BM25-EXTRACTED-TREC-QUERIES            |       0.043  |        0.0479 |      0.0286 |       0.0326 |
| BM25-META+EXTRACTED+NODES-ALL-QUERIES  |       0.4095 |        0.4446 |      0.2563 |       0.33   |
| BM25-META+EXTRACTED+NODES-SYN-QUERIES  |       0.2299 |        0.2442 |

Synthetic row analysis

In [13]:
def find_row(markdown_table: str, target_name: str) -> str:
    lines = markdown_table.strip().split("\n")

    for line in lines:
        if re.match(f"\| {re.escape(target_name)}", line):
            return line

In [14]:
synthetic_rows = [
    "CS-META+EXTRACTED-ALL-QUERIES",
    "CS-META+EXTRACTED+NODES-ALL-QUERIES",
    "CS-META-ALL-QUERIES",
    "CS-META+NODES-ALL-QUERIES",
    "CS-EXTRACTED-ALL-QUERIES",
    "CS-NODES-ALL-QUERIES",
    "CS-EXTRACTED+NODES-ALL-QUERIES",
    "BM25-META+EXTRACTED-ALL-QUERIES",
    "BM25-META+EXTRACTED+NODES-ALL-QUERIES",
    "BM25-META-ALL-QUERIES",
    "BM25-META+NODES-ALL-QUERIES",
    "BM25-EXTRACTED-ALL-QUERIES",
    "BM25-NODES-ALL-QUERIES",
    "BM25-EXTRACTED+NODES-ALL-QUERIES",
    "LMD-META+EXTRACTED-ALL-QUERIES",
    "LMD-META+EXTRACTED+NODES-ALL-QUERIES",
    "LMD-META-ALL-QUERIES",
    "LMD-META+NODES-ALL-QUERIES",
    "LMD-EXTRACTED-ALL-QUERIES",
    "LMD-NODES-ALL-QUERIES",
    "LMD-EXTRACTED+NODES-ALL-QUERIES"
]

for name in synthetic_rows:
    print(find_row(markdown_table, name))

| CS-META+EXTRACTED-ALL-QUERIES          |       0.4474 |        0.4648 |      0.2865 |       0.3608 |
| CS-META+EXTRACTED+NODES-ALL-QUERIES    |       0.4475 |        0.4708 |      0.2868 |       0.3668 |
| CS-META-ALL-QUERIES                    |       0.4247 |        0.4363 |      0.2681 |       0.3371 |
| CS-META+NODES-ALL-QUERIES              |       0.4296 |        0.4447 |      0.2722 |       0.3451 |
| CS-EXTRACTED-ALL-QUERIES               |       0.078  |        0.0878 |      0.0483 |       0.0572 |
| CS-NODES-ALL-QUERIES                   |       0.0849 |        0.0826 |      0.0417 |       0.0459 |
| CS-EXTRACTED+NODES-ALL-QUERIES         |       0.1048 |        0.1129 |      0.0564 |       0.0667 |
| BM25-META+EXTRACTED-ALL-QUERIES        |       0.4387 |        0.4749 |      0.2805 |       0.3633 |
| BM25-META+EXTRACTED+NODES-ALL-QUERIES  |       0.4095 |        0.4446 |      0.2563 |       0.33   |
| BM25-META-ALL-QUERIES                  |       0.4616 |        0.4808 |